In [1]:
import pandas as pd 
import re
import numpy as np

In [2]:

df1=pd.read_csv("recipes.csv") 
df2=pd.read_csv("recipes1.csv") 
df3=pd.read_csv("recipes2.csv") 
df4=pd.read_csv("recipes3.csv")

In [3]:
#combines the data from two scrapped csvs (the webscrapping was done in two shifts) 
#drops dulicate rows where a recipe appeared in >1 search


print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
recipedf= pd.concat([df1,df2,df3,df4],ignore_index=True)
print(recipedf.shape)
recipedf=recipedf.drop('Unnamed: 0',axis=1)
recipedf=recipedf.drop_duplicates()
print(recipedf.shape)


(2007, 6)
(1198, 6)
(699, 6)
(276, 6)
(4180, 6)
(2838, 5)


In [4]:
#gets rid of recipe titles that are irrelevant- must be a cake and be vanilla/white/yellow


recipedf=recipedf[recipedf["Recipe_title"].str.contains("Cake|cake")& recipedf["Recipe_title"].str.contains("Vanilla|Yellow|White|Angel|Rainbow|Almond|Lemon|Confetti|Sprinkles|Coconut|Funfetti|Sour|Pound")]
#recipedf[recipedf["Recipe_title"].str.contains("A|B")==True]
recipedf.shape

(557, 5)

In [5]:
recipedf.columns
servings_per_tin={9:12,8:10,6:8,10:14}
print(servings_per_tin.get(int(re.search('\((.*)-inch','(9-inch)').group(1))))

12


In [9]:
def cleanservings(servings):
    
# """"
# built to translate different strings regarding serving sizes into floats. references 
# https://larkcakeshop.com/wp-content/uploads/2018/05/CakeServeGuide2.pdf when deciding how much 
# a x-inch cake serves
# """"
    regex = re.compile('\(.+-inch\)')
    regex1 = re.compile('.+-inch') 
    servings=str(servings)
    servings_per_tin={9:12,8:10,6:8,10:14}
    
    if "cupcakes" in servings:
        try:
            servings=int(servings[-11:-9])
        except:
            servings=int(re.search('\d+', servings).group(0))
    
    elif bool(re.match(regex, servings)):
        try:
            servings=servings_per_tin.get(int(re.search('\((.*)-inch',servings).group(1))) #[int(servings.split(" ")[0]),re.search('\((.*)-inch',servings).group(1)]
        #except:
            #try: 
                
            #servings=servings_per_tin.get(int(re.search('\((.*)-inch','servings').group(1))) #[servings.split(" ")[0].lower().index(['one','two','three','four'])+1,re.search('\((.*)-inch',recipe.servings).group(1)]
        except:
            servings=servings
    elif bool(re.match(regex1, servings)):
        try:
            servings=servings_per_tin.get(int(re.search(' (.*)-inch',servings).group(1)))#[['one','two','three','four'].index(servings.split(" ")[0].lower())+1,int(re.search(' (.*)-inch',servings).group(1))]
            #servings=[int(servings.split(" ")[0]),int(re.search(' (.*)-inch',servings).group(1))]
        #except:
            #try: 
                #servings=servings_per_tin.get(int(re.search(' (.*)-inch',servings).group(1)))#[['one','two','three','four'].index(servings.split(" ")[0].lower())+1,int(re.search(' (.*)-inch',servings).group(1))]
        except:
            servings==servings
    elif servings=='1 cake':
        servings=12
    elif servings=='2 cakes':
        servings=24
    elif servings=='1 loaf':
        servings=8
    elif servings=='2 loaves':
        servings=16
    else:
        servings=re.sub(r' \(.*\)', '',servings)
                
        if "servings" in servings:
            servings=servings[:-9]
            try:
                servings=int(servings)
            except:
                try:
                    servings=(int(servings.split(" ")[0])+int(servings.split(" ")[2]))/2
                except:
                    try:
                        servings=(int(servings.split("-")[0])+int(servings.split("-")[1]))/2
                    except:
                        servings=servings
        else:
            try:
                servings=int(re.search('\d+', servings).group(0))
            except:
                servings=servings
            
                
    return servings

In [10]:
recipedf.servings=recipedf.servings.map(lambda a: cleanservings(a))

In [11]:
def cleaningredients(ing):

    ing=re.sub(r' \(.*\)', '',ing).replace('[', '').replace(']', '').replace('\', ', '').replace('/xa0', '').split('\'')[1:-1]
    def fractofloat(x):
        #converts the / fractions to decimals
        try:
            return re.sub(r'(\d+)/(\d+)', str(int(re.findall(r"(\d+)/", x)[0])/int(re.findall(r"/(\d+)", x)[0])),x)
        except: 
            return x
    ing=[fractofloat(x) for x in ing]
    
    

In [204]:
# ing=re.sub(r' \(.*\)', '',recipedf.ingredients[0]).replace('[', '').replace(']', '').replace('\', ', '').replace('/xa0', '').split('\'')[1:-1]
def fractofloat(x):
        #converts the / fractions to decimals
        try:
            return re.sub(r'(\d+)/(\d+)', str(int(re.findall(r"(\d+)/", x)[0])/int(re.findall(r"/(\d+)", x)[0])),x)
        except: 
            return x
# ing=[fractofloat(x) for x in ing]


def parseingredients(x):
    
    """
    3 0.5 cups flour->
    3.5=measnum
    cups=measunit
    flour=corring
    
    """
    #print(x)
    if type((re.search('\d+', x)))!=type(re.search('.\d+', 'ggf')):
        measnum=float(re.search('\d+', x).group(0))
        if measnum!=0:
            x=x.replace(re.search('\d+', x).group(0), '')
    else:
        #x=x.replace(re.search('\d+', x).group(0), '')
        measnum=1
        x='1 '+x
    if type(re.search('0.\d+', x))!=type(re.search('.\d+', 'ggf')):
        
        measnum=measnum+float(re.search('0.\d+', x).group(0))
        x=x.replace(re.search('0.\d+', x).group(0), '')
    measunit=x.split()[0]
    corring=x.replace(x.split()[0],'')
    return [measnum,measunit,corring]
def simplifying(corring):
    
    """"shortens long winded ingredient descriptions 
    ie unsalted butter, at room temperature-> butter"""
    basicingredients=['flour','cake flour','sugar', 'butter', 'vanilla extract','almond extract','lemon extract','baking soda','baking powder','eggs','egg','oil','milk','water','cocoa powder','berries','box','salt','confectioners sugar','vanilla','powdered sugar','spray','brown sugar','nuts','chocolate chips','lemon juice']
    
    #notin=np.zeros(len(basicingredients))
    for basicing in basicingredients:
        print(basicing)
        try:
            corring[[(basicing in el) for el in corring].index(True)]=basicing
        except:
             #print(corring)
            u=1
    return corring




In [205]:
def cleaningingredients(ing):
    if len(ing)<=2:
        return ''
    ing=re.split('\\\'',re.sub('\([^()]*\)', ' ',ing.replace('confectioners\' sugar','confectioners sugar').replace('to','').replace('\'s','').replace('[', '').replace(']', '').replace('\',', '').replace('\\xa0', ' ').replace(' \"','\'')))[1:-1]
    #print(ing)
    ing=[fractofloat(x) for x in ing]    
    ing=[parseingredients(x) for x in ing]
    [measnum,measunit,corring]=(list(zip(*ing)))
    corring=list(corring)
    measunit=list(measunit)
    measnum=list(measnum)
    corring=simplifying(corring)
    return list(zip(measnum,measunit,corring))


In [206]:

# print(x)
#cleaningingredients(recipedf.ingredients[10])

In [207]:

for i in range(10):
    print(cleaningingredients(recipedf.ingredients[i]))

flour
cake flour
sugar
butter
vanilla extract
almond extract
lemon extract
baking soda
baking powder
eggs
egg
oil
milk
water
cocoa powder
berries
box
salt
confectioners sugar
vanilla
powdered sugar
spray
brown sugar
nuts
chocolate chips
lemon juice
[(1.0, 'stick', 'butter'), (2.75, 'cups', 'flour'), (1.0, 'teaspoon', 'baking powder'), (1.0, 'teaspoon', 'salt'), (2.0, 'cups', 'sugar'), (0.5, 'cup', 'oil'), (5.0, 'large', 'egg'), (1.0, 'tablespoon', 'vanilla'), (1.0, 'cup', 'milk')]
flour
cake flour
sugar
butter
vanilla extract
almond extract
lemon extract
baking soda
baking powder
eggs
egg
oil
milk
water
cocoa powder
berries
box
salt
confectioners sugar
vanilla
powdered sugar
spray
brown sugar
nuts
chocolate chips
lemon juice
[(1, '1', 'spray'), (3.5, 'cups', 'flour'), (2.0, 'tablespoons', '  cornstarch '), (2.0, 'teaspoons', 'baking powder'), (2.0, 'teaspoons', 'salt'), (1.0, 'teaspoon', 'baking soda'), (2.0, 'sticks', 'butter'), (2.0, 'cups', 'sugar'), (3.0, 'tablespoons', 'oil'), (1.

In [187]:
(recipedf.ingredients[5])

'[\'Nonstick cooking spray, for greasing the pans\', \'3 cups all-purpose flour, plus 1/4 cup for dusting the pans\\xa0\', \'1 tablespoon baking powder\\xa0\', \'1/2 teaspoon salt\\xa0\', \'2 sticks unsalted butter, at room temperature\', \'1 1/4 cups granulated sugar\\xa0\', \'4 large eggs, at room temperature\\xa0\', \'1 tablespoon vanilla extract\\xa0\', \'1 1/4 cups whole milk (or 3/4 cup heavy cream mixed with 1/2 cup water)\\xa0\', "4 cups confectioners\' sugar", \'1 cup (2 sticks) unsalted butter\\xa0\', \'2 teaspoons vanilla extract\\xa0\', \'1 to 2 tablespoons whipping cream, plus additional if needed\\xa0\']'

In [210]:
'salt' in '   Pinch of salt '

True